In [1]:
from agno.embedder.ollama import OllamaEmbedder
from agno.models.ollama import Ollama
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
from agno.vectordb.lancedb import LanceDb, SearchType
from agno.agent import Agent


In [2]:
embedder = OllamaEmbedder(id="mxbai-embed-large", dimensions=1024)

In [3]:
vector_db = LanceDb(
    table_name="gloomhaven",
    uri="/tmp/lancedb",
    search_type=SearchType.hybrid,
    embedder=embedder
)

In [4]:
knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://cdn.1j1ju.com/medias/8d/c5/21-gloomhaven-rulebook.pdf"],
    vector_db=vector_db
)

In [5]:
knowledge_base.load(upsert=True)

INFO Loading knowledge base

INFO Reading: https://cdn.1j1ju.com/medias/8d/c5/21-gloomhaven-rulebook.pdf

INFO Skipped 55 existing/duplicate documents.

INFO Added 0 documents to knowledge base

In [8]:
agent = Agent(
    model=Ollama(id="gemma3:4b", options={"num_ctx": 16192}),
    # Enable RAG
    knowledge=knowledge_base,
    add_context=True,
    # Add references to the original documents
    add_references=True,
    description="You are an expert in the rules of the board game gloomhaven.",
    instructions=[
        "Use additional data provided for the corresponding rules.",
        "Cite the rules book with the corresponding information at the end of the answer to a question"
    ],
    search_knowledge=False,
    add_history_to_messages=True,
    num_history_responses=10,
    markdown=True,
    # debug_mode=True,
)

In [9]:
prompt = "Summarize game rules in three sentences"
response = agent.run(prompt)
print(response.content)

INFO Found 5 documents

Okay, here’s a summary of the Gloomhaven rules in three sentences, incorporating the information from the rulebook:

Gloomhaven is a cooperative tactical board game where players take on the roles of mercenaries, each with unique abilities, attempting to complete scenario objectives. Each round, players secretly select cards to perform actions – moving, attacking, and utilizing special abilities – determined by an initiative order revealed at the start of the round. Players strategically coordinate to overcome monster encounters and achieve scenario goals, managing exhaustion and carefully utilizing their limited hand of cards to succeed.


In [10]:
prompt = "How many players can play at once?"
response = agent.run(prompt)
print(response.content)

INFO Found 5 documents

Okay, let’s answer your question about how many players can play at once in Gloomhaven:

According to the rulebook, Gloomhaven can be played with **1 to 4 players** simultaneously.
